### Análisis de modelos para la predicción de partidos

Imports

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.utils import shuffle

## 4 TEMPORADAS

# Obtención de los datos

In [ ]:

#Devuelve el índice en la lista del nombre del equipo introducido
def getindexteam(team, classification):
  i = 0
  for t in classification:
    if t[0] == team:
      return i
    else:
      i = i+1

def getdatafromcsv(filename):

  df = pd.DataFrame(columns =['Home', 'Away', 'pointsH', 'pointsA',
                            'avgTotalHgoals', 'avgTotalAgoals', 'avgTotalHconcededgoals','avgTotalAconcededgoals',
                            'avgHgoalslocal', 'avgAgoalslocal', 'avgHgoalsconcededlocal','avgAgoalsconcededlocal', 
                            'avgHgoalsaway', 'avgAgoalsaway', 'avgHgoalsconcededaway','avgAgoalsconcededaway',
                            'localstreak','awaystreak',
                            'Result'])

  #archive = files.upload()
  datacsv = pd.read_csv(filename, sep=";")
  #display(datacsv.head())

  #columnas:  0 nombre, 1 puntos en clasificacion,  2 partidos jugados como local,  3 partidos jugados como visitante, 
  #           4 goles a favor como local, 5 goles en contra como local, 6 goels a favor como visitante, 7 goles en contra como visitante
  classification = []

  for i in range(10):
    classification.append([datacsv.loc[i][0], 0, 0, 0, 0, 0, 0, 0])
    classification.append([datacsv.loc[i][1], 0, 0, 0, 0, 0, 0, 0])

  a = []

  for i in range(len(datacsv)):

    localteam = datacsv.loc[i][0] #nombre equipo lcoal
    awayteam = datacsv.loc[i][1] #nombre equipo visitante
    indexlocalteam = getindexteam(localteam, classification)
    indexawayteam = getindexteam(awayteam, classification)

    pointsH = classification[indexlocalteam][1] #puntos clasificacion local
    pointsA = classification[indexawayteam][1] #puntos clasificacion visitante

    avgHgoalslocal = 0 # media goles del equipo local como  local
    avgAgoalslocal = 0 #media goles del equipo vistante como local
    avgHgoalsaway = 0 # media goles del equipo local como  visitante
    avgAgoalsaway = 0 # media goles del equipo visitante como  visitante
    if classification[indexlocalteam][4]!=0 or classification[indexlocalteam][2] !=0:
      avgHgoalslocal = classification[indexlocalteam][4] / classification[indexlocalteam][2]
    if classification[indexawayteam][4]!=0 or classification[indexawayteam][2] !=0:
      avgAgoalslocal = classification[indexawayteam][4] / classification[indexawayteam][2]
    if classification[indexlocalteam][6]!=0 or classification[indexlocalteam][3] !=0:
      avgHgoalsaway = classification[indexlocalteam][6] / classification[indexlocalteam][3]
    if classification[indexawayteam][6]!=0 or classification[indexawayteam][3] !=0:
      avgAgoalsaway = classification[indexawayteam][6] / classification[indexawayteam][3]

    avgTotalHgoals = (avgHgoalslocal + avgHgoalsaway)/2 #media goles totales equipo local
    avgTotalAgoals = (avgAgoalslocal + avgAgoalsaway)/2 #media goles totales equipo visitante

    avgHgoalsconcededlocal = 0 # media goles encajados del equipo local como  local
    avgAgoalsconcededlocal = 0 #media goles encajados del equipo vistante como local
    avgHgoalsconcededaway = 0 # media goles encajados del equipo local como  visitante
    avgAgoalsconcededaway = 0 # media goles encajados del equipo visitante como  visitante
    if classification[indexlocalteam][5]!=0 or classification[indexlocalteam][2] !=0:
      avgHgoalsconcededlocal = classification[indexlocalteam][5] / classification[indexlocalteam][2]
    if classification[indexawayteam][5]!=0 or classification[indexawayteam][2] !=0:
      avgAgoalsconcededlocal = classification[indexawayteam][5] / classification[indexawayteam][2]
    if classification[indexlocalteam][7]!=0 or classification[indexlocalteam][3] !=0:
      avgHgoalsconcededaway = classification[indexlocalteam][7] / classification[indexlocalteam][3]
    if classification[indexawayteam][7]!=0 or classification[indexawayteam][3] !=0:
      avgAgoalsconcededaway = classification[indexawayteam][7] / classification[indexawayteam][3]

    avgTotalHconcededgoals = (avgHgoalsconcededlocal + avgHgoalsconcededaway)/2 #media goles encajados totales equipo local
    avgTotalAconcededgoals = (avgAgoalsconcededlocal + avgAgoalsconcededaway)/2 #media goles encajados totales equipo visitante

    rangelocal = 0
    localstreak = 0
    aux = df[df.Home == localteam]
    aux1 = df[df.Away == localteam]
    aux = pd.concat([aux,aux1])
    aux = aux.sort_index()
    aux = aux.iloc[::-1]
    
    if len(aux) >= 5:
      rangelocal = 5
    else:
      rangelocal = len(aux)
    for t in range(rangelocal):
      if aux.iloc[t,0] == localteam:
        if aux.iloc[t,18] == "H":
          localstreak += 3
        elif aux.iloc[t,18] == "D":
          localstreak += 1
      elif aux.iloc[t,1] == localteam:
        if aux.iloc[t,18] == "A":
          localstreak += 3
        elif aux.iloc[t,18] == "D":
          localstreak += 1
 
    rangeaway = 0
    awaystreak = 0
    aux = df[df.Home == awayteam]
    aux1 = df[df.Away == awayteam]
    aux = pd.concat([aux,aux1])
    aux = aux.sort_index()
    aux = aux.iloc[::-1]

    if len(aux) >= 5:
      rangeaway = 5
    else:
      rangeaway = len(aux)
    for t in range(rangeaway):
      if aux.iloc[t,0] == awayteam:
        if aux.iloc[t,18] == "H":
          awaystreak += 3
        elif aux.iloc[t,18] == "D":
          awaystreak += 1
      elif aux.iloc[t,1] == awayteam:
        if aux.iloc[t,18] == "A":
          awaystreak += 3
        elif aux.iloc[t,18] == "D":
          awaystreak += 1

    result = ""
    if datacsv.loc[i][2] > datacsv.loc[i][3]:
      classification[indexlocalteam][1] += 3
      result = "H"
    elif datacsv.loc[i][2] == datacsv.loc[i][3]:
      classification[indexlocalteam][1] += 1
      classification[indexawayteam][1] += 1
      result = "D"
    else: 
      classification[indexawayteam][1] += 3
      result = "A"

    a = [localteam,awayteam,pointsH,pointsA,
         avgTotalHgoals,avgTotalAgoals,avgTotalHconcededgoals,avgTotalAconcededgoals,
         avgHgoalslocal,avgAgoalslocal,avgHgoalsconcededlocal,avgAgoalsconcededlocal,
         avgHgoalsaway,avgAgoalsaway,avgHgoalsconcededaway,avgAgoalsconcededaway,
         localstreak,awaystreak,result]

    df.loc[len(df)] = a

    classification[indexlocalteam][2] += 1
    classification[indexlocalteam][4] += datacsv.loc[i][2]
    classification[indexlocalteam][5] += datacsv.loc[i][3]
    classification[indexawayteam][3] += 1
    classification[indexawayteam][6] += datacsv.loc[i][3]
    classification[indexawayteam][7] += datacsv.loc[i][2]

  return df


dftotal = pd.DataFrame(columns =['Home', 'Away', 'pointsH', 'pointsA',
                            'avgTotalHgoals', 'avgTotalAgoals', 'avgTotalHconcededgoals','avgTotalAconcededgoals',
                            'avgHgoalslocal', 'avgAgoalslocal', 'avgHgoalsconcededlocal','avgAgoalsconcededlocal', 
                            'avgHgoalsaway', 'avgAgoalsaway', 'avgHgoalsconcededaway','avgAgoalsconcededaway',
                            'localstreak','awaystreak',
                            'Result'])

df1 = getdatafromcsv("LaLigaData1516.csv")
df2 = getdatafromcsv("LaLigaData1617.csv")
df3 = getdatafromcsv("LaLigaData1718.csv")
df4 = getdatafromcsv("LaLigaData1819.csv")

dftotal = pd.concat([df1, df2, df3, df4])

matches = len(dftotal)
homewins = len(dftotal[dftotal.Result == 'H'])
awaywins = len(dftotal[dftotal.Result == 'A'])
draws = len(dftotal[dftotal.Result == 'D'])
print("Número de partidos " +str(matches))
print("Número victorias equipos locales " + str(homewins))
print("Número empates " + str(draws))
print("Número victorias equipos vistante " + str(awaywins))

# Experimento 1

In [5]:
def exp1(features):

  x = features[['pointsH','pointsA','avgTotalHgoals','avgTotalAgoals','avgTotalHconcededgoals','avgTotalAconcededgoals','localstreak','awaystreak']]
  y = features[['Result']]
    
  numbercrossv = 5

  clf = LogisticRegression()
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  y_pred = cross_val_predict(clf, x, y.values.ravel(), cv=numbercrossv)
  data = confusion_matrix(y, y_pred)
  df_cm = pd.DataFrame(data, columns=np.unique(y), index = np.unique(y))
  df_cm.index.name = 'Actual'
  df_cm.columns.name = 'Predicted'
  plt.figure(figsize = (5,4))
  plt.title("Regresión logística")
  sn.set(font_scale=0.8)
  sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16}, fmt='d')
  metricas.iloc[0,1] = scores.mean()

  clf = SVC(kernel='linear')
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  y_pred = cross_val_predict(clf, x, y.values.ravel(), cv=numbercrossv)
  data = confusion_matrix(y, y_pred)
  df_cm = pd.DataFrame(data, columns=np.unique(y), index = np.unique(y))
  df_cm.index.name = 'Actual'
  df_cm.columns.name = 'Predicted'
  plt.figure(figsize = (5,4))
  plt.title("SVM con kernel lineal")
  sn.set(font_scale=0.8)
  sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16}, fmt='d')
  metricas.iloc[1,1] = scores.mean()

  clf = SVC(kernel='poly')
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  y_pred = cross_val_predict(clf, x, y.values.ravel(), cv=numbercrossv)
  data = confusion_matrix(y, y_pred)
  df_cm = pd.DataFrame(data, columns=np.unique(y), index = np.unique(y))
  df_cm.index.name = 'Actual'
  df_cm.columns.name = 'Predicted'
  plt.figure(figsize = (5,4))
  plt.title("SVM con kernel polinomial")
  sn.set(font_scale=0.8)
  sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16}, fmt='d')
  metricas.iloc[2,1] = scores.mean()

  clf = KNeighborsClassifier()
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  y_pred = cross_val_predict(clf, x, y.values.ravel(), cv=numbercrossv)
  data = confusion_matrix(y, y_pred)
  df_cm = pd.DataFrame(data, columns=np.unique(y), index = np.unique(y))
  df_cm.index.name = 'Actual'
  df_cm.columns.name = 'Predicted'
  plt.figure(figsize = (5,4))
  plt.title("KNN")
  sn.set(font_scale=0.8)
  sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16}, fmt='d')
  metricas.iloc[3,1] = scores.mean()

  clf = GaussianNB()
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  y_pred = cross_val_predict(clf, x, y.values.ravel(), cv=numbercrossv)
  data = confusion_matrix(y, y_pred)
  df_cm = pd.DataFrame(data, columns=np.unique(y), index = np.unique(y))
  df_cm.index.name = 'Actual'
  df_cm.columns.name = 'Predicted'
  plt.figure(figsize = (5,4))
  plt.title("Gaussian Naive Bayes")
  sn.set(font_scale=0.8)
  sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16}, fmt='d')
  metricas.iloc[4,1] = scores.mean()

  clf = RandomForestClassifier()
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  y_pred = cross_val_predict(clf, x, y.values.ravel(), cv=numbercrossv)
  data = confusion_matrix(y, y_pred)
  df_cm = pd.DataFrame(data, columns=np.unique(y), index = np.unique(y))
  df_cm.index.name = 'Actual'
  df_cm.columns.name = 'Predicted'
  plt.figure(figsize = (5,4))
  plt.title("Random Forest")
  sn.set(font_scale=0.8)
  sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16}, fmt='d')
  metricas.iloc[5,1] = scores.mean()

  clf = SVC(kernel = "rbf")
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  y_pred = cross_val_predict(clf, x, y.values.ravel(), cv=numbercrossv)
  data = confusion_matrix(y, y_pred)
  df_cm = pd.DataFrame(data, columns=np.unique(y), index = np.unique(y))
  df_cm.index.name = 'Actual'
  df_cm.columns.name = 'Predicted'
  plt.figure(figsize = (5,4))
  plt.title("SVM con kernel RBF")
  sn.set(font_scale=0.8)#for label size
  sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16}, fmt='d')
  #print(confusion_matrix(y, y_pred))
  metricas.iloc[6,1] = scores.mean()
    
  clf = MLPClassifier()
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  y_pred = cross_val_predict(clf, x, y.values.ravel(), cv=numbercrossv)
  data = confusion_matrix(y, y_pred)
  df_cm = pd.DataFrame(data, columns=np.unique(y), index = np.unique(y))
  df_cm.index.name = 'Actual'
  df_cm.columns.name = 'Predicted'
  plt.figure(figsize = (5,4))
  plt.title("Multilayer Perceptron")
  sn.set(font_scale=0.8)#for label size
  sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16}, fmt='d')
  #print(confusion_matrix(y, y_pred))
  metricas.iloc[7,1] = scores.mean()
  
  probhome = len(dftotal[dftotal.Result == 'H'])/ len(dftotal)
  probdraw = len(dftotal[dftotal.Result == 'D'])/ len(dftotal)
  probaway = len(dftotal[dftotal.Result == 'A'])/ len(dftotal)
  y_pred_random = []
  for i in range(len(x)):
    y_pred_random.append(np.random.choice(["H", "D", "A"],p=[probhome,probdraw,probaway]))
  metricas.iloc[8,1] = accuracy_score(y, y_pred_random)
  data = confusion_matrix(y, y_pred)
  df_cm = pd.DataFrame(data, columns=np.unique(y), index = np.unique(y))
  df_cm.index.name = 'Actual'
  df_cm.columns.name = 'Predicted'
  plt.figure(figsize = (5,4))
  plt.title("Random model")
  sn.set(font_scale=0.8)#for label size
  sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16}, fmt='d')
  #print(confusion_matrix(y, y_pred_random))
  plt.clf()
  plt.close()

  display(metricas.head(10))

  plt.figure(figsize=(9, 5))
  plt.ylabel('Accuracy')
  index = np.arange(len(metricas['model']))
  plt.xticks(index, metricas['model'], fontsize=10, rotation=90)
  plt.ylim(0.3, 0.55)
  plt.bar(metricas['model'], metricas['accuracy'])
  plt.show()

Utilización del dataset con muestras desbalanceadas

In [ ]:
metricas = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", 
                                   "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                        'accuracy': [0,0,0,0,0,0,0,0,0]})

features = dftotal[['pointsH','pointsA','avgTotalHgoals','avgTotalAgoals','avgTotalHconcededgoals','avgTotalAconcededgoals',
                    'localstreak','awaystreak','Result']]
features = features.drop_duplicates()

print(display(features.tail()))
matches = len(features)
homewins = len(features[features.Result == 'H'])
awaywins = len(features[features.Result == 'A'])
draws = len(features[features.Result == 'D'])
print("Número de partidos " +str(matches))
print("Número victorias equipos locales " + str(homewins))
print("Número empates " + str(draws))
print("Número victorias equipos vistante " + str(awaywins))

exp1(features)

Utilización del dataset con muestras balanceadas

In [ ]:
metricas = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", 
                                   "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                        'accuracy': [0,0,0,0,0,0,0,0,0]})

features = dftotal[['pointsH','pointsA','avgTotalHgoals','avgTotalAgoals','avgTotalHconcededgoals','avgTotalAconcededgoals',
                    'localstreak','awaystreak','Result']]
features = features.drop_duplicates()

draws = len(features[features.Result == 'D'])
hwins = len(features[features.Result == 'H'])
awins = len(features[features.Result == 'A'])
homewinsw = features[features.Result == 'H'][(hwins-draws):hwins]
awaywinsw = features[features.Result == 'A'][(awins-draws):awins]
drawsw = features[features.Result == 'D']
features = shuffle(pd.concat([homewinsw, awaywinsw,drawsw]))
features = features.reset_index(drop=True)

print(display(features.tail()))
matches = len(features)
homewins = len(features[features.Result == 'H'])
awaywins = len(features[features.Result == 'A'])
draws = len(features[features.Result == 'D'])
print("Número de partidos " +str(matches))
print("Número victorias equipos locales " + str(homewins))
print("Número empates " + str(draws))
print("Número victorias equipos vistante " + str(awaywins))

exp1(features)

## 2 TEMPORADAS

# Obtención de los datos

In [ ]:
#Devuelve el índice en la lista del nombre del equipo introducido
def getindexteam(team, classification):
  i = 0
  for t in classification:
    if t[0] == team:
      return i
    else:
      i = i+1

def getdatafromcsv(filename):

  df = pd.DataFrame(columns =['Home', 'Away', 'pointsH', 'pointsA',
                            'avgTotalHgoals', 'avgTotalAgoals', 'avgTotalHconcededgoals','avgTotalAconcededgoals',
                            'avgHgoalslocal', 'avgAgoalslocal', 'avgHgoalsconcededlocal','avgAgoalsconcededlocal', 
                            'avgHgoalsaway', 'avgAgoalsaway', 'avgHgoalsconcededaway','avgAgoalsconcededaway',
                            'xglocal','xgaway','xglocalconceded','xgawayconceded',
                            'avgHxglocal', 'avgAxglocal', 'avgHxgconcededlocal','avgAxgconcededlocal', 
                            'avgHxgaway', 'avgAxgaway', 'avgHxgconcededaway','avgAxgconcededaway',
                            'localstreak','awaystreak', 'especial',
                            'mgh','mga','mah','maa','persavH','persavA',
                            'Result'])

  #archive = files.upload()
  datacsv = pd.read_csv(filename, sep=";")

  #columnas:  
  #0 nombre,  1 puntos en clasificacion,        2 partidos jugados como local,  3 partidos jugados como visitante, 
  #           4 goles a favor como local,       5 goles en contra como local,   6 goels a favor como visitante, 
  #           7 goles en contra como visitante, 8 xg a favor como local,          9 xg en contra como local, 
  #           10 xg a favor como visitante,     11 xg en contra como visitante    12 porcentaje paradas portero

  classification = []

  for i in range(10):
    classification.append([datacsv.loc[i][0], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    classification.append([datacsv.loc[i][1], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

  a = []

  for i in range(len(datacsv)):

    localteam = datacsv.loc[i][0] #nombre equipo lcoal
    awayteam = datacsv.loc[i][1] #nombre equipo visitante
    indexlocalteam = getindexteam(localteam, classification)
    indexawayteam = getindexteam(awayteam, classification)

    pointsH = classification[indexlocalteam][1] #puntos clasificacion local
    pointsA = classification[indexawayteam][1] #puntos clasificacion visitante

    avgHgoalslocal = 0 # media goles del equipo local como  local
    avgAgoalslocal = 0 #media goles del equipo vistante como local
    avgHgoalsaway = 0 # media goles del equipo local como  visitante
    avgAgoalsaway = 0 # media goles del equipo visitante como  visitante
    if classification[indexlocalteam][4]!=0 or classification[indexlocalteam][2] !=0:
      avgHgoalslocal = classification[indexlocalteam][4] / classification[indexlocalteam][2]
    if classification[indexawayteam][4]!=0 or classification[indexawayteam][2] !=0:
      avgAgoalslocal = classification[indexawayteam][4] / classification[indexawayteam][2]
    if classification[indexlocalteam][6]!=0 or classification[indexlocalteam][3] !=0:
      avgHgoalsaway = classification[indexlocalteam][6] / classification[indexlocalteam][3]
    if classification[indexawayteam][6]!=0 or classification[indexawayteam][3] !=0:
      avgAgoalsaway = classification[indexawayteam][6] / classification[indexawayteam][3]

    avgTotalHgoals = (avgHgoalslocal + avgHgoalsaway)/2 #media goles totales equipo local
    avgTotalAgoals = (avgAgoalslocal + avgAgoalsaway)/2 #media goles totales equipo visitante

    avgHgoalsconcededlocal = 0 # media goles encajados del equipo local como  local
    avgAgoalsconcededlocal = 0 #media goles encajados del equipo vistante como local
    avgHgoalsconcededaway = 0 # media goles encajados del equipo local como  visitante
    avgAgoalsconcededaway = 0 # media goles encajados del equipo visitante como  visitante
    if classification[indexlocalteam][5]!=0 or classification[indexlocalteam][2] !=0:
      avgHgoalsconcededlocal = classification[indexlocalteam][5] / classification[indexlocalteam][2]
    if classification[indexawayteam][5]!=0 or classification[indexawayteam][2] !=0:
      avgAgoalsconcededlocal = classification[indexawayteam][5] / classification[indexawayteam][2]
    if classification[indexlocalteam][7]!=0 or classification[indexlocalteam][3] !=0:
      avgHgoalsconcededaway = classification[indexlocalteam][7] / classification[indexlocalteam][3]
    if classification[indexawayteam][7]!=0 or classification[indexawayteam][3] !=0:
      avgAgoalsconcededaway = classification[indexawayteam][7] / classification[indexawayteam][3]

    avgTotalHconcededgoals = (avgHgoalsconcededlocal + avgHgoalsconcededaway)/2 #media goles encajados totales equipo local
    avgTotalAconcededgoals = (avgAgoalsconcededlocal + avgAgoalsconcededaway)/2 #media goles encajados totales equipo visitante

    rangelocal = 0
    localstreak = 0
    aux = df[df.Home == localteam]
    aux1 = df[df.Away == localteam]
    aux = pd.concat([aux,aux1])
    aux = aux.sort_index()
    aux = aux.iloc[::-1]
    
    if len(aux) >= 5:
      rangelocal = 5
    else:
      rangelocal = len(aux)
    for t in range(rangelocal):
      if aux.iloc[t,0] == localteam:
        if aux.iloc[t,37] == "H":
          localstreak += 3
        elif aux.iloc[t,37] == "D":
          localstreak += 1
      elif aux.iloc[t,1] == localteam:
        if aux.iloc[t,37] == "A":
          localstreak += 3
        elif aux.iloc[t,37] == "D":
          localstreak += 1
 
    rangeaway = 0
    awaystreak = 0
    aux = df[df.Home == awayteam]
    aux1 = df[df.Away == awayteam]
    aux = pd.concat([aux,aux1])
    aux = aux.sort_index()
    aux = aux.iloc[::-1]

    if len(aux) >= 5:
      rangeaway = 5
    else:
      rangeaway = len(aux)
    for t in range(rangeaway):
      if aux.iloc[t,0] == awayteam:
        if aux.iloc[t,37] == "H":
          awaystreak += 3
        elif aux.iloc[t,37] == "D":
          awaystreak += 1
      elif aux.iloc[t,1] == awayteam:
        if aux.iloc[t,37] == "A":
          awaystreak += 3
        elif aux.iloc[t,37] == "D":
          awaystreak += 1


    xgHlocal = 0 # xg del equipo local como  local
    xgAlocal = 0 #xg del equipo vistante como local
    xgHaway = 0 # xg del equipo local como  visitante
    xgAaway = 0 # xg del equipo visitante como  visitante
    avgxgHgoals = 0 #media xg totales equipo local
    avgxgAgoals = 0 #media xg totales equipo visitante

    if classification[indexlocalteam][8]!=0 or classification[indexlocalteam][2] !=0:
      xgHlocal = classification[indexlocalteam][8] / classification[indexlocalteam][2]
    if classification[indexawayteam][8]!=0 or classification[indexawayteam][2] !=0:
      xgAlocal = classification[indexawayteam][8] / classification[indexawayteam][2]
    if classification[indexlocalteam][10]!=0 or classification[indexlocalteam][3] !=0:
      xgHaway = classification[indexlocalteam][10] / classification[indexlocalteam][3]
    if classification[indexawayteam][10]!=0 or classification[indexawayteam][3] !=0:
      xgAaway = classification[indexawayteam][10] / classification[indexawayteam][3]

    avgxgHgoals = (xgHlocal + xgHaway)/2 
    avgxgAgoals = (xgAlocal + xgAaway)/2 

    xgHlocalconceded = 0 # xg encajados del equipo local como  local
    xgAlocalconceded = 0 # xg encajados del equipo vistante como local
    xgHawayconceded = 0 # xg encajados del equipo local como  visitante
    xgAawayconceded = 0 # xg encajados del equipo visitante como  visitante
    avgTotalHxgconceded = 0#media xg encajados totales equipo local
    avgTotalAxgconceded = 0#media xg encajados totales equipo visitante

    if classification[indexlocalteam][9]!=0 or classification[indexlocalteam][2] !=0:
      xgHlocalconceded = classification[indexlocalteam][9]/ classification[indexlocalteam][2]
    if classification[indexawayteam][9]!=0 or classification[indexawayteam][2] !=0:
      xgAlocalconceded = classification[indexawayteam][9] / classification[indexawayteam][2]
    if classification[indexlocalteam][11]!=0 or classification[indexlocalteam][3] !=0:
      xgHawayconceded = classification[indexlocalteam][11] / classification[indexlocalteam][3]
    if classification[indexawayteam][11]!=0 or classification[indexawayteam][3] !=0:
      xgAawayconceded = classification[indexawayteam][11] / classification[indexawayteam][3]

    avgTotalHxgconceded = (xgHlocalconceded + xgHawayconceded)/2 
    avgTotalAxgconceded = (xgAlocalconceded + xgAawayconceded)/2 

    especial = datacsv.loc[i][6] #partido especial
    mgh = datacsv.loc[i][7] #disponibilidad máximo goleador local
    mga = datacsv.loc[i][8] #disponibilidad máximo goleador visitante
    mah = datacsv.loc[i][9] #disponibilidad máximo asistente local
    maa = datacsv.loc[i][10] #disponibilidad máximo asistente visitante

    persaveH = 0 #porcentaje paradas portero local
    persaveA = 0 #porcentaje paradas portero visitante
    if classification[indexlocalteam][12]!=0 and (classification[indexlocalteam][2] !=0 or classification[indexlocalteam][3] !=0):
      persaveH = classification[indexlocalteam][12]/ (classification[indexlocalteam][2] + classification[indexlocalteam][3])
    if classification[indexawayteam][12]!=0 and (classification[indexawayteam][2] !=0 or classification[indexawayteam][3] !=0):
      persaveA = classification[indexawayteam][12] / (classification[indexlocalteam][2] + classification[indexlocalteam][3])

    result = ""
    if datacsv.loc[i][2] > datacsv.loc[i][3]:
      classification[indexlocalteam][1] += 3
      result = "H"
    elif datacsv.loc[i][2] == datacsv.loc[i][3]:
      classification[indexlocalteam][1] += 1
      classification[indexawayteam][1] += 1
      result = "D"
    else: 
      classification[indexawayteam][1] += 3
      result = "A"
    
    a = [localteam,awayteam,pointsH,pointsA,
         avgTotalHgoals,avgTotalAgoals,avgTotalHconcededgoals,avgTotalAconcededgoals,
         avgHgoalslocal,avgAgoalslocal,avgHgoalsconcededlocal,avgAgoalsconcededlocal,
         avgHgoalsaway,avgAgoalsaway,avgHgoalsconcededaway,avgAgoalsconcededaway,
         avgxgHgoals,avgxgAgoals,avgTotalHxgconceded,avgTotalAxgconceded,
         xgHlocal,xgAlocal,xgHlocalconceded,xgAlocalconceded,
         xgHaway,xgAaway,xgHawayconceded,xgAawayconceded,
         localstreak,awaystreak,especial,
         mgh,mga,mah,maa,persaveH,persaveA,result]

    df.loc[len(df)] = a

    classification[indexlocalteam][2] += 1
    classification[indexlocalteam][4] += datacsv.loc[i][2]
    classification[indexlocalteam][5] += datacsv.loc[i][3]
    classification[indexawayteam][3] += 1
    classification[indexawayteam][6] += datacsv.loc[i][3]
    classification[indexawayteam][7] += datacsv.loc[i][2]
    classification[indexlocalteam][8] += datacsv.loc[i][4]
    classification[indexlocalteam][9] += datacsv.loc[i][5]
    classification[indexawayteam][10] += datacsv.loc[i][5]
    classification[indexawayteam][11] += datacsv.loc[i][4]
    classification[indexlocalteam][12] += datacsv.loc[i][11]
    classification[indexawayteam][12] += datacsv.loc[i][12]

  return df


df = pd.DataFrame(columns =['Home', 'Away', 'pointsH', 'pointsA',
                            'avgTotalHgoals', 'avgTotalAgoals', 'avgTotalHconcededgoals','avgTotalAconcededgoals',
                            'avgHgoalslocal', 'avgAgoalslocal', 'avgHgoalsconcededlocal','avgAgoalsconcededlocal', 
                            'avgHgoalsaway', 'avgAgoalsaway', 'avgHgoalsconcededaway','avgAgoalsconcededaway',
                            'xglocal','xgaway','xglocalconceded','xgawayconceded',
                            'avgHxglocal', 'avgAxglocal', 'avgHxgconcededlocal','avgAxgconcededlocal', 
                            'avgHxgaway', 'avgAxgaway', 'avgHxgconcededaway','avgAxgconcededaway',
                            'localstreak','awaystreak', 'especial',
                            'mgh','mga','mah','maa','persavH','persavA',
                            'Result'])


df1 = getdatafromcsv("LaLigaData1718.csv")
df2 = getdatafromcsv("LaLigaData1819.csv")
dftotal = pd.concat([df1, df2])

matches = len(dftotal)
homewins = len(dftotal[dftotal.Result == 'H'])
awaywins = len(dftotal[dftotal.Result == 'A'])
draws = len(dftotal[dftotal.Result == 'D'])
print("Número de partidos " +str(matches))
print("Número victorias equipos locales " + str(homewins))
print("Número empates " + str(draws))
print("Número victorias equipos vistante " + str(awaywins))
print(display(dftotal.tail()))

dftotal['pointsH']=dftotal['pointsH'].astype('int')
dftotal['pointsA']=dftotal['pointsA'].astype('int')
dftotal['avgHgoalslocal']=dftotal['avgHgoalslocal'].astype('float')
dftotal['avgAgoalslocal']=dftotal['avgAgoalslocal'].astype('float')
dftotal['avgHgoalsconcededlocal']=dftotal['avgHgoalsconcededlocal'].astype('float')
dftotal['avgAgoalsconcededlocal']=dftotal['avgAgoalsconcededlocal'].astype('float')
dftotal['avgHgoalsaway']=dftotal['avgHgoalsaway'].astype('float')
dftotal['avgAgoalsaway']=dftotal['avgAgoalsaway'].astype('float')
dftotal['avgHgoalsconcededaway']=dftotal['avgHgoalsconcededaway'].astype('float')
dftotal['avgAgoalsconcededaway']=dftotal['avgAgoalsconcededaway'].astype('float')
dftotal['avgHxglocal']=dftotal['avgHxglocal'].astype('float')
dftotal['avgAxglocal']=dftotal['avgAxglocal'].astype('float')
dftotal['avgHxgconcededlocal']=dftotal['avgHxgconcededlocal'].astype('float')
dftotal['avgAxgconcededlocal']=dftotal['avgAxgconcededlocal'].astype('float')
dftotal['avgHxgaway']=dftotal['avgHxgaway'].astype('float')
dftotal['avgAxgaway']=dftotal['avgAxgaway'].astype('float')
dftotal['avgHxgconcededaway']=dftotal['avgHxgconcededaway'].astype('float')
dftotal['avgAxgconcededaway']=dftotal['avgAxgconcededaway'].astype('float')
dftotal['localstreak']=dftotal['localstreak'].astype('int')
dftotal['awaystreak']=dftotal['awaystreak'].astype('int')
dftotal['especial']=dftotal['especial'].astype('int')
dftotal['mgh']=dftotal['mgh'].astype('int')
dftotal['mga']=dftotal['mga'].astype('int')
dftotal['mah']=dftotal['mah'].astype('int')
dftotal['maa']=dftotal['maa'].astype('int')
dftotal['persavH']=dftotal['persavH'].astype('float')
dftotal['persavA']=dftotal['persavA'].astype('float')

In [10]:
features = dftotal[['pointsH', 'pointsA','avgTotalHgoals', 'avgTotalAgoals', 'avgTotalHconcededgoals','avgTotalAconcededgoals',
                    'avgHgoalslocal', 'avgAgoalslocal', 'avgHgoalsconcededlocal','avgAgoalsconcededlocal', 'avgHgoalsaway', 'avgAgoalsaway', 
                    'avgHgoalsconcededaway','avgAgoalsconcededaway','xglocal','xgaway','xglocalconceded','xgawayconceded',
                    'avgHxglocal', 'avgAxglocal', 'avgHxgconcededlocal','avgAxgconcededlocal', 'avgHxgaway', 'avgAxgaway', 'avgHxgconcededaway',
                    'avgAxgconcededaway','localstreak','awaystreak', 'especial','mgh','mga','mah','maa','persavH','persavA','Result']]

features = features.drop_duplicates()
homewinsw = features[features.Result == 'H']
homewinsw = homewinsw.drop(homewinsw[homewinsw['pointsH']<=4].index)
homewinsw = homewinsw.drop(homewinsw[homewinsw['pointsA']<=4].index)
homewinsw = shuffle(homewinsw)
awaywinsw = features[features.Result == 'A']
awaywinsw = awaywinsw.drop(awaywinsw[awaywinsw['pointsH']<=4].index)
awaywinsw = awaywinsw.drop(awaywinsw[awaywinsw['pointsA']<=4].index)
awaywinsw = shuffle(awaywinsw)
drawsw = features[features.Result == 'D']
drawsw = drawsw.drop(drawsw[drawsw['pointsH']<=4].index)
drawsw = drawsw.drop(drawsw[drawsw['pointsA']<=4].index)
drawsw = shuffle(drawsw)

wins = len(homewinsw[homewinsw.Result == 'H'])
loses = len(awaywinsw[awaywinsw.Result == 'A'])
draws = len(drawsw[drawsw.Result == 'D'])

features = pd.concat([homewinsw[0:draws], awaywinsw[0:draws],drawsw])
features = shuffle(features)
features = features.reset_index(drop=True)

wins = len(features[features.Result == 'H'])
loses = len(features[features.Result == 'A'])
draws = len(features[features.Result == 'D'])
print(wins,loses,draws)

xtotal = features.drop(['Result'],1)
y = features[['Result']]


164 164 164


In [ ]:
def traintest(x, y):

  metricas = pd.DataFrame(columns =['Home'])

  numbercrossv = 5

  clf = LogisticRegression()
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  metricas.loc[len(metricas)] = scores.mean()

  clf = SVC(kernel='linear')
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  metricas.loc[len(metricas)] = scores.mean()

  clf = SVC(kernel='poly')
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  metricas.loc[len(metricas)] = scores.mean()

  clf = KNeighborsClassifier()
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  metricas.loc[len(metricas)] = scores.mean()

  clf = GaussianNB()
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  metricas.loc[len(metricas)] = scores.mean()

  clf = RandomForestClassifier()
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  metricas.loc[len(metricas)] = scores.mean()

  clf = SVC(kernel = "rbf")
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  metricas.loc[len(metricas)] = scores.mean()
    
  clf = MLPClassifier()
  scores = cross_val_score(clf, x, y.values.ravel(), cv=numbercrossv)
  metricas.loc[len(metricas)] = scores.mean()
  
  probhome = len(dftotal[dftotal.Result == 'H'])/ len(dftotal)
  probdraw = len(dftotal[dftotal.Result == 'D'])/ len(dftotal)
  probaway = len(dftotal[dftotal.Result == 'A'])/ len(dftotal)
  y_pred_random = []
  for i in range(len(x)):
    y_pred_random.append(np.random.choice(["H", "D", "A"],p=[probhome,probdraw,probaway]))
  metricas.loc[len(metricas)] = accuracy_score(y, y_pred_random)

  return metricas

# Experimento 2

Comparativa diferentes características para expresar los goles

In [ ]:

avgmetricas1 = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                            'accuracy': [0,0,0,0,0,0,0,0,0]
                            })
avgmetricas2 = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                            'accuracy': [0,0,0,0,0,0,0,0,0]
                            })
avgmetricas3 = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                            'accuracy': [0,0,0,0,0,0,0,0,0]
                            })
avgmetricas4 = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                            'accuracy': [0,0,0,0,0,0,0,0,0]
                            })

x = xtotal[['pointsH','pointsA','avgTotalHgoals','avgTotalAgoals','avgTotalHconcededgoals','avgTotalAconcededgoals',
            'avgHgoalslocal','avgAgoalsaway','avgHgoalsconcededlocal','avgAgoalsconcededaway','xglocal','xgaway','xglocalconceded','xgawayconceded',
            'avgHxglocal', 'avgAxgaway','avgHxgconcededlocal', 'avgAxgconcededaway','localstreak','awaystreak']]

sn.heatmap(x.corr(),annot=True,cmap='RdYlGn',linewidths=0.1)
fig=plt.gcf()
fig.set_size_inches(40,24)
plt.show()

x_exp1 = x[['pointsH','pointsA','avgTotalHgoals','avgTotalAgoals','avgTotalHconcededgoals','avgTotalAconcededgoals','localstreak','awaystreak']]
metricas = traintest(x_exp1,y)
for r in range(9):
    avgmetricas1.iloc[r,1] += metricas.iloc[r,0]

x_exp2 = x[['pointsH','pointsA','xglocal','xgaway','xglocalconceded','xgawayconceded','localstreak','awaystreak']]
metricas = traintest(x_exp2,y)
for r in range(9):
    avgmetricas2.iloc[r,1] += metricas.iloc[r,0]  

x_exp3 = x[['pointsH','pointsA','avgHgoalslocal','avgAgoalsaway','avgHgoalsconcededlocal','avgAgoalsconcededaway','localstreak','awaystreak']]
metricas = traintest(x_exp3,y)
for r in range(9):
    avgmetricas3.iloc[r,1] += metricas.iloc[r,0]
  
x_exp4 = x[['pointsH','pointsA','avgHxglocal', 'avgAxgaway','avgHxgconcededlocal', 'avgAxgconcededaway','localstreak','awaystreak']]
metricas = traintest(x_exp4,y)
for r in range(9):
    avgmetricas4.iloc[r,1] += metricas.iloc[r,0]


print("Gráfico comparativo del accuracy según los tipos de expresión de los goles")
dfdraw = pd.concat([ avgmetricas1.iloc[0:8]['accuracy'], avgmetricas2.iloc[0:8]['accuracy'], 
                    avgmetricas3.iloc[0:8]['accuracy'], avgmetricas4.iloc[0:8]['accuracy']], axis=1)
dfdraw.columns = ['Goles totales', 'xG totales', 'Goles específicos', 'xG específicos']
display(dfdraw.head(10))

drawavgmetricas1 = avgmetricas1.iloc[0:8]
drawavgmetricas2 = avgmetricas2.iloc[0:8]
drawavgmetricas3 = avgmetricas3.iloc[0:8]
drawavgmetricas4 = avgmetricas4.iloc[0:8]
plt.figure(figsize=(10, 2))
xw = np.arange(len(drawavgmetricas1['model']))  # the label locations
width = 0.20  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(xw - width *1.5, drawavgmetricas1['accuracy'], width, label='Datos goles totales')
rects2 = ax.bar(xw - width/2, drawavgmetricas2['accuracy'], width, label='Datos xG',color='red')
rects3 = ax.bar(xw + width/2, drawavgmetricas3['accuracy'], width, label='Datos goles específicos',color='green')
rects4 = ax.bar(xw + width*1.5, drawavgmetricas4['accuracy'], width, label='Datos xG específicos',color='orange')

dftotal.iloc[0:5]
ax.set_ylabel('Accuracy')
ax.set_xticks(xw)
ax.set_xticklabels(drawavgmetricas2['model'],rotation=90)
plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left")
sn.set()
plt.ylim(0.35, 0.47)

fig.tight_layout()
plt.show()

Combinaciones entre tipos

In [ ]:
avgmetricas0 = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                            'accuracy': [0,0,0,0,0,0,0,0,0]
                            })
avgmetricas1 = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                            'accuracy': [0,0,0,0,0,0,0,0,0]
                            })
avgmetricas2 = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                            'accuracy': [0,0,0,0,0,0,0,0,0]
                            })
avgmetricas3 = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                            'accuracy': [0,0,0,0,0,0,0,0,0]
                            })
avgmetricas4 = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                            'accuracy': [0,0,0,0,0,0,0,0,0]
                            })


x_exp0 = x[['pointsH','pointsA','xglocal','xgaway','xglocalconceded','xgawayconceded','localstreak','awaystreak']]
metricas = traintest(x_exp0,y)
for r in range(9):
    avgmetricas0.iloc[r,1] += metricas.iloc[r,0]

x_exp1 = x[['pointsH','pointsA','avgTotalHgoals','avgTotalAgoals','avgTotalHconcededgoals','avgTotalAconcededgoals',
              'xglocal','xgaway','xglocalconceded','xgawayconceded','localstreak','awaystreak']]
metricas = traintest(x_exp1,y)
for r in range(9):
      avgmetricas1.iloc[r,1] += metricas.iloc[r,0]

x_exp2 = x[['pointsH','pointsA','avgTotalHgoals','avgTotalAgoals','avgTotalHconcededgoals','avgTotalAconcededgoals',
              'avgHxglocal', 'avgAxgaway','avgHxgconcededlocal', 'avgAxgconcededaway','localstreak','awaystreak']]
metricas = traintest(x_exp2,y)
for r in range(9):
  avgmetricas2.iloc[r,1] += metricas.iloc[r,0]  

x_exp3 = x[['pointsH','pointsA','avgHgoalslocal','avgAgoalsaway','avgHgoalsconcededlocal','avgAgoalsconcededaway',
            'xglocal','xgaway','xglocalconceded','xgawayconceded','localstreak','awaystreak']]
metricas = traintest(x_exp3,y)
for r in range(9):
    avgmetricas3.iloc[r,1] += metricas.iloc[r,0]
  
x_exp4 = x[['pointsH','pointsA','avgHgoalslocal','avgAgoalsaway','avgHgoalsconcededlocal','avgAgoalsconcededaway',
            'avgHxglocal', 'avgAxgaway','avgHxgconcededlocal', 'avgAxgconcededaway','localstreak','awaystreak']]
metricas = traintest(x_exp4,y)
for r in range(9):
    avgmetricas4.iloc[r,1] += metricas.iloc[r,0]


print("Gráfico comparativo del accuracy según las combinaciones realizadas")
dfdraw = pd.concat([avgmetricas0.iloc[0:8]['accuracy'], avgmetricas1.iloc[0:8]['accuracy'], avgmetricas2.iloc[0:8]['accuracy'], 
                    avgmetricas3.iloc[0:8]['accuracy'], avgmetricas4.iloc[0:8]['accuracy']], axis=1)
dfdraw.columns = ['xG totales','Goles totales + xG totales', 'Goles totales + xG específicos', 
                  'xG específicos + Goles totales', 'xG específicos + Goles específicos']
display(dfdraw.head(10))


# Experimento 3

In [ ]:

avgmetricas1 = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                            'accuracy': [0,0,0,0,0,0,0,0,0]
                            })
avgmetricas2 = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                            'accuracy': [0,0,0,0,0,0,0,0,0]
                            })
avgmetricas3 = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                            'accuracy': [0,0,0,0,0,0,0,0,0]
                            })
avgmetricas4 = pd.DataFrame({'model': ["Logistic Regression", "Linear kernel SVM", "Polynomial kernel SVM","KNN", "Gaussian Naive Bayes", "Random Forest Classifier", "RBF kernel SVM", "NN Multilayer Perceptron","Random Model"], 
                            'accuracy': [0,0,0,0,0,0,0,0,0]
                            })

x = xtotal[['pointsH', 'pointsA','xglocal','xgaway','xglocalconceded','xgawayconceded','localstreak','awaystreak', 'especial',
            'mgh','mga','mah','maa','persavH','persavA']]

display(x.head(10))

x_exp1 = x[['pointsH','pointsA','xglocal','xgaway','xglocalconceded','xgawayconceded',
            'localstreak','awaystreak']]
metricas = traintest(x_exp1,y)
for r in range(9):
    avgmetricas1.iloc[r,1] += metricas.iloc[r,0]

x_exp2 = x[['pointsH','pointsA','xglocal','xgaway','xglocalconceded','xgawayconceded', 'mgh','mga','mah','maa','persavH','persavA',
            'localstreak','awaystreak']]
metricas = traintest(x_exp2,y)
for r in range(9):
    avgmetricas2.iloc[r,1] += metricas.iloc[r,0]  

x_exp3 = x[['pointsH','pointsA','xglocal','xgaway','xglocalconceded','xgawayconceded',
            'localstreak','awaystreak','especial']]
metricas = traintest(x_exp3,y)
for r in range(9):
    avgmetricas3.iloc[r,1] += metricas.iloc[r,0]
  
x_exp4 = x[['pointsH','pointsA','xglocal','xgaway','xglocalconceded','xgawayconceded', 'mgh','mga','mah','maa','persavH','persavA','especial',
            'localstreak','awaystreak']]
metricas = traintest(x_exp4,y)
for r in range(9):
    avgmetricas4.iloc[r,1] += metricas.iloc[r,0]


dfdraw = pd.concat([avgmetricas1.iloc[0:8]['accuracy'], avgmetricas2.iloc[0:8]['accuracy'], 
                    avgmetricas3.iloc[0:8]['accuracy'], avgmetricas4.iloc[0:8]['accuracy']], axis=1)
dfdraw.columns = ['Caracteristicas del exp 2','Características exp2 + especial', 'Características exp2 + jugadores', 
                  'Todas las características']
display(dfdraw.head(10))

drawavgmetricas1 = avgmetricas1.iloc[0:8]
drawavgmetricas2 = avgmetricas2.iloc[0:8]
drawavgmetricas3 = avgmetricas3.iloc[0:8]
drawavgmetricas4 = avgmetricas4.iloc[0:8]
plt.figure(figsize=(10, 2))
xw = np.arange(len(drawavgmetricas1['model']))  # the label locations
width = 0.20  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(xw - width *1.5, drawavgmetricas1['accuracy'], width, label='Datos goles totales')
rects2 = ax.bar(xw - width/2, drawavgmetricas2['accuracy'], width, label='Datos xG',color='red')
rects3 = ax.bar(xw + width/2, drawavgmetricas3['accuracy'], width, label='Datos goles específicos',color='green')
rects4 = ax.bar(xw + width*1.5, drawavgmetricas4['accuracy'], width, label='Datos xG específicos',color='orange')

dftotal.iloc[0:5]
ax.set_ylabel('Accuracy')
ax.set_xticks(xw)
ax.set_xticklabels(drawavgmetricas2['model'],rotation=90)
plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left")
sn.set()
plt.ylim(0.35, 0.50)

fig.tight_layout()
plt.show()